In [18]:
import pandas as pd
import os, glob, csv, re
import gzip
import shutil

In [62]:
#data=pd.read_csv('../ORTHOLOGY-ALLIANCE_COMBINED.tsv.gz')

infile = os.path.join("c:\\Users\\19802\\Documents\\dev\scripps\\BioThings\\ORTHOLOGY-ALLIANCE_COMBINED.tsv.gz")
assert os.path.exists(infile)


## Load the data to inspect and setup parser    

Datasource: https://fms.alliancegenome.org/download/ORTHOLOGY-ALLIANCE_COMBINED.tsv.gz   
The first lines (0-14) are summary information and are disregarded when formatting the dataframe.

In [69]:
with gzip.open(infile, 'rb') as f_in:
    #summary=pd.read_csv(f_in)
    data=pd.read_csv(f_in, header=15, sep="\\t").to_dict(orient='records')



C:\Users\19802\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Now that we have loaded the data and correctly formatted it into records, we can loop through

In [70]:
for rec in data[:4]:
    print(rec)

{'Gene1ID': 'WB:WBGene00011502', 'Gene1Symbol': 'vps-53', 'Gene1SpeciesTaxonID': 'NCBITaxon:6239', 'Gene1SpeciesName': 'Caenorhabditis elegans', 'Gene2ID': 'SGD:S000003566', 'Gene2Symbol': 'VPS53', 'Gene2SpeciesTaxonID': 'NCBITaxon:559292', 'Gene2SpeciesName': 'Saccharomyces cerevisiae', 'Algorithms': 'PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector', 'AlgorithmsMatch': 9, 'OutOfAlgorithms': 10, 'IsBestScore': 'Yes', 'IsBestRevScore': 'Yes'}
{'Gene1ID': 'ZFIN:ZDB-GENE-041114-199', 'Gene1Symbol': 'vps53', 'Gene1SpeciesTaxonID': 'NCBITaxon:7955', 'Gene1SpeciesName': 'Danio rerio', 'Gene2ID': 'SGD:S000003566', 'Gene2Symbol': 'VPS53', 'Gene2SpeciesTaxonID': 'NCBITaxon:559292', 'Gene2SpeciesName': 'Saccharomyces cerevisiae', 'Algorithms': 'OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector', 'AlgorithmsMatch': 8, 'OutOfAlgorithms': 10, 'IsBestScore': 'Yes', 'IsBestRevScore': 'Yes'}
{'Gene1ID': 'RGD:1311391', 'Gen

In [68]:
data

[{'Gene1ID': 'WB:WBGene00011502',
  'Gene1Symbol': 'vps-53',
  'Gene1SpeciesTaxonID': 'NCBITaxon:6239',
  'Gene1SpeciesName': 'Caenorhabditis elegans',
  'Gene2ID': 'SGD:S000003566',
  'Gene2Symbol': 'VPS53',
  'Gene2SpeciesTaxonID': 'NCBITaxon:559292',
  'Gene2SpeciesName': 'Saccharomyces cerevisiae',
  'Algorithms': 'PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector',
  'AlgorithmsMatch': 9,
  'OutOfAlgorithms': 10,
  'IsBestScore': 'Yes',
  'IsBestRevScore': 'Yes'},
 {'Gene1ID': 'ZFIN:ZDB-GENE-041114-199',
  'Gene1Symbol': 'vps53',
  'Gene1SpeciesTaxonID': 'NCBITaxon:7955',
  'Gene1SpeciesName': 'Danio rerio',
  'Gene2ID': 'SGD:S000003566',
  'Gene2Symbol': 'VPS53',
  'Gene2SpeciesTaxonID': 'NCBITaxon:559292',
  'Gene2SpeciesName': 'Saccharomyces cerevisiae',
  'Algorithms': 'OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roundup|InParanoid|PANTHER|OrthoInspector',
  'AlgorithmsMatch': 8,
  'OutOfAlgorithms': 10,
  'IsBestScore': 'Yes',
  'IsBe

In [65]:
data.columns.values

array(['Gene1ID', 'Gene1Symbol', 'Gene1SpeciesTaxonID',
       'Gene1SpeciesName', 'Gene2ID', 'Gene2Symbol',
       'Gene2SpeciesTaxonID', 'Gene2SpeciesName', 'Algorithms',
       'AlgorithmsMatch', 'OutOfAlgorithms', 'IsBestScore',
       'IsBestRevScore'], dtype=object)

In [66]:
data.head()

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,WB:WBGene00011502,vps-53,NCBITaxon:6239,Caenorhabditis elegans,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
1,ZFIN:ZDB-GENE-041114-199,vps53,NCBITaxon:7955,Danio rerio,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|Roun...,8,10,Yes,Yes
2,RGD:1311391,Vps53,NCBITaxon:10116,Rattus norvegicus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,OrthoFinder|Hieranoid|OMA|Ensembl Compara|InPa...,7,9,Yes,Yes
3,MGI:1915549,Vps53,NCBITaxon:10090,Mus musculus,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes
4,HGNC:25608,VPS53,NCBITaxon:9606,Homo sapiens,SGD:S000003566,VPS53,NCBITaxon:559292,Saccharomyces cerevisiae,PhylomeDB|OrthoFinder|Hieranoid|OMA|Ensembl Co...,9,10,Yes,Yes


---